<h1><center>Home Credit Risk Prediction</center></h1>
<center>November 2024</center>
<center>Celine Ng</center>

# Table of Contents

1. Project Introduction   
    1. Notebook Preparation
    1. Data loading
1. Main Data Preparation
    1. Data cleaning
    1. Dataframes and keys
    1. Main data preparation
1. Initial Data Cleaning
    1. Duplicate rows
    1. Datatypes
    1. Missing values
    1. Values
1. EDA
    1. Correlation
    1. Statistical Inference
    1. Distribution
1. Data Preprocessing
1. Feature Selection
    1. All features included
    1. Mutual Information
    1. PCA
1. Models
    1. Baseline model
    1. Basic model
    1. Hyperparameter Tuning
    1. Test Data
    1. Final Model
    1. Deployment
    1. Model Interpretation
1. Improvements

# 1. Project Introduction

**Crucial problem for retail banks** <br>
1. Minimize loan defaults by evaluating credit risk accurately.
2. Maximize profits by better identifying customers that are NOT 
currently handed out loans but are potentially reliable.


**Project Objective**<br>
The second problem is not solvable with provided data. So the focus of this 
project would be the following:<br>
1. Improve risk evaluation accuracy to retail banks. In practice
 meaning target variable classification.
2. Evaluate feature importance to explain decisions.
3. Provide actionable insights to improve credit scoring.

**Initial Plan**<br>
1. Data cleaning (missing values counting) of each table
2. Select important tables
3. Aggregate table information - 1 row per person instead of loan
4. Join columns to main table
5. EDA and statistical inference
6. Feature engineering - New feature creations with domain knowledge
7. Feature selection
8. Modeling 
9. Hyperparameter tuning
10. Evaluation with cross validation
11. Deployment
12. Model interpretation

## 1.1. Notebook Preparation

In [ ]:
%%capture
%pip install -r requirements.txt

In [ ]:
from IPython.display import HTML
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import os
from utils.eda import *
from utils.model import *
from utils.stats import *
from utils.data_preparation import *


from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

## 1.2. Data Loading

Objective: Brief overview of our datasets, including the features and target 
variable

The data comes with 10 separate CSV files. It is originally based on a Kaggle
 competition that is now closed, 
[Home Credit Default Risk](https://www.kaggle.com/competitions/home-credit-default-risk/overview).

<ol>
<li>application_{train|test}.csv

This is the main table, broken into two files for Train (with TARGET) and Test (without TARGET).
Static data for all applications. One row represents one loan in our data 
sample.</li>
<li>bureau.csv

All client's previous credits provided by other financial institutions that were reported to Credit Bureau (for clients who have a loan in our sample).
For every loan in our sample, there are as many rows as number of credits 
the client had in Credit Bureau before the application date.</li>
<li>bureau_balance.csv

Monthly balances of previous credits in Credit Bureau.
This table has one row for each month of history of every previous credit 
reported to Credit Bureau – i.e the table has (#loans in sample * # of 
relative previous credits * # of months where we have some history 
observable for the previous credits) rows.</li>
<li>POS_CASH_balance.csv

Monthly balance snapshots of previous POS (point of sales) and cash loans that the applicant had with Home Credit.
This table has one row for each month of history of every previous credit in
 Home Credit (consumer credit and cash loans) related to loans in our sample
  – i.e. the table has (#loans in sample * # of relative previous credits 
  *# of months in which we have some history observable for the previous 
  credits) rows.</li>
<li>credit_card_balance.csv

Monthly balance snapshots of previous credit cards that the applicant has with Home Credit.
This table has one row for each month of history of every previous credit in
 Home Credit (consumer credit and cash loans) related to loans in our sample
  – i.e. the table has (#loans in sample * # of relative previous credit 
  cards *# of months where we have some history observable for the previous
   credit card) rows.</li>
<li>previous_application.csv

All previous applications for Home Credit loans of clients who have loans in our sample.
There is one row for each previous application related to loans in our data 
sample.</li>
<li>installments_payments.csv

Repayment history for the previously disbursed credits in Home Credit related to the loans in our sample.
There is a) one row for every payment that was made plus b) one row each for missed payment.
One row is equivalent to one payment of one installment OR one installment 
corresponding to one payment of one previous Home Credit credit related to 
loans in our sample.</li>

**Adjacent csv files:**
<li>HomeCredit_columns_description.csv

This file contains descriptions for the columns in the various data files.</li>

<li>sample submission.csv
Sample submission file for Kaggle competition</li>
</ol>

*Non-adjacent csv files will be converted into pkl files for more efficient 
memory usage. To do so, please run 'convert_csv_to_pkl.py'.*

**Home Credit columns description**

In [ ]:
description = pd.read_csv('data_csv/HomeCredit_columns_description.csv', encoding='latin1')
display(description.head())
description_shape = description.shape
print(f"Number of rows on bureau data_csv: {description_shape[0]}\nNumber of "
      f"columns on bureau data_csv: {description_shape[1]}")

In [ ]:
description.iloc[1, 3]

In [ ]:
description.loc[description['Row'] == 'SK_ID_CURR', 'Description']

**Application train**

In [ ]:
application_train = pd.read_pickle('data_pkl/application_train.pkl')
display(application_train.head())
application_train_shape = application_train.shape
print(f"Number of rows on train data_csv: {application_train_shape[0]}\nNumber of "
      f"columns on train data_csv: {application_train_shape[1]}")

**Application Test**

In [ ]:
application_test = pd.read_pickle('data_pkl/application_test.pkl')
display(application_test.head())
application_test_shape = application_test.shape
print(f"Number of rows on train data_csv: {application_test_shape[0]}\nNumber of "
      f"columns on train data_csv: {application_test_shape[1]}")

For both application train and test, SK_ID_CURR is the key that identifies 
each row in the table.

**Bureau**

In [ ]:
bureau = pd.read_pickle('data_pkl/bureau.pkl')
display(bureau.head())
bureau_shape = bureau.shape
print(f"Number of rows on bureau data_csv: {bureau_shape[0]}\nNumber of "
      f"columns on bureau data_csv: {bureau_shape[1]}")

**Bureau Balance**

In [ ]:
bureau_balance = pd.read_pickle('data_pkl/bureau_balance.pkl')
display(bureau_balance.head())
bureau_balance_shape = bureau_balance.shape
print(f"Number of rows on bureau data_csv: {bureau_balance_shape[0]}\nNumber of "
      f"columns on bureau data_csv: {bureau_balance_shape[1]}")

**Previous Application**

In [ ]:
previous_application = pd.read_pickle('data_pkl/previous_application.pkl')
display(previous_application.head())
previous_application_shape = previous_application.shape
print(f"Number of rows on bureau data_csv: {previous_application_shape[0]}\nNumber of "
      f"columns on bureau data_csv: {previous_application_shape[1]}")

**POS CASH balance**

In [ ]:
POS_CASH_balance = pd.read_pickle('data_pkl/POS_CASH_balance.pkl')
display(POS_CASH_balance.head())
POS_CASH_balance_shape = POS_CASH_balance.shape
print(f"Number of rows on bureau data_csv: {POS_CASH_balance_shape[0]}\nNumber of "
      f"columns on bureau data_csv: {POS_CASH_balance_shape[1]}")

**Installments Payments**

In [ ]:
installments_payments = pd.read_pickle('data_pkl/installments_payments.pkl')
display(installments_payments.head())
installments_payments_shape = installments_payments.shape
print(f"Number of rows on bureau data_csv: {installments_payments_shape[0]}\nNumber of "
      f"columns on bureau data_csv: {installments_payments_shape[1]}")

**Credit Card Balance**

In [ ]:
credit_card_balance = pd.read_pickle('data_pkl/credit_card_balance.pkl')
display(credit_card_balance.head())
credit_card_balance_shape = credit_card_balance.shape
print(f"Number of rows on bureau data_csv: {credit_card_balance_shape[0]}\nNumber of"
      f"columns on bureau data_csv: {credit_card_balance_shape[1]}")

# 2. Main Data Preparation
Objective:
In order to aggregate data to main data tables, a quick exploration is 
needed to understand which data are useful and how should each be aggregated
. These data will also need to be split into train and test before the 
aggregation, as the main table is already split.

## 2.1. Data Cleaning
Objective: Initial brief data cleaning to check what data is usable for 
further analysis.

**Duplicates**

In [ ]:
application_test.duplicated().any()

In [ ]:
application_train.duplicated().any()

In [ ]:
sk_id_curr_train = application_train['SK_ID_CURR']
sk_id_curr_test = application_test['SK_ID_CURR']

duplicate_ids = set(sk_id_curr_train).intersection(set(sk_id_curr_test))
duplicate_ids

No duplicates, so each table has unique loan ID, SK_ID_CURR. Meaning 
SK_ID_CURR from application tables will help distinguish loans for training 
and loans for test.

**Check for Missing Values** -<br>
There is a possibility of abandoning certain tables if they contained too many 
missing values.

In [ ]:
dataframes = {
      'application_train': application_train, 
      'application_test': application_test, 
      'bureau': bureau,
      'bureau_balance': bureau_balance,
      'credit_card_balance': credit_card_balance, 
      'installments_payments': installments_payments,
      'POS_CASH_balance': POS_CASH_balance, 
      'previous_application': previous_application
}

for df_name, df in dataframes.items():
      print(df_name)
      display(missing_values(df).sort_values(ascending=False, by='Missing '
                                                                 'Values'))

There are some features with large amount of missing values. However, no one
 table has too many missing values where it needs to be discarded as a whole
 .<br>
 For modeling, tree based and non tree based models like logistic 
 regression, naive bayes, random forest, XGBoost, and LightGBM will be 
 tested. Non tree based models require complete data, while imputation and 
 feature removal will still be considered when >40% of missing data for tree 
 based models.

## 2.2. Dataframes and Keys

In [ ]:
keys_to_check = ['SK_ID_CURR', 'SK_ID_PREV', 'SK_ID_BUREAU']
results = []

for table_name, df in dataframes.items():
    row = {'Table': table_name, 'Total_Rows': len(df)}
    
    for key in keys_to_check:
        row[key] = df[key].nunique() if key in df.columns else None
    results.append(row)

key_counts_df = pd.DataFrame(results)
key_counts_df

Total SK_ID_CURR, loan IDs : 356255<br>
Total SK_ID_BUREAU, bureau loan IDs : 1716428, from 305811 loan IDs. 
About 14.16% of SK_ID_CURR do not have bureau info.<br>
Total SK_ID_PREV, previous loan IDs : 1670214, from 338857 loan IDs
. About 4.88% of SK_ID_CURR do not have previous loan info.<br><br>
    
**Tables with unique keys:**<br>
    1. application_train: SK_ID_CURR<br>
    2. application_test: SK_ID_CURR <br>
    3. bureau: SK_ID_BUREAU. <i>Each SK_ID_CURR can correspond to 
    several SK_ID_BUREAU, and not all SK_ID_CURR are present in this 
    dataframe.</i><br>
    4. previous_application: SK_ID_PREV. <i>Each SK_ID_CURR can correspond to 
    several SK_ID_PREV, and not all SK_ID_CURR are present in this 
    dataframe.</i><br><br>
    
**Tables without unique keys:**<br>
    1. bureau_balance: SK_ID_BUREAU corresponds to several rows. Not all 
    SK_ID_BUREAU are present in this dataframe.<br>
    2. credit_card_balance: SK_ID_PREV corresponds to several rows. Not all 
    SK_ID_PREV are present in this dataframe.<br>
    3. installments_payments: SK_ID_PREV corresponds to several rows. Not 
    all SK_ID_PREV are present in this dataframe.<br>
    4. POS_CASH_balance: SK_ID_PREV corresponds to several rows. Not all 
    SK_ID_PREV are present in this dataframe.<br><br>

## 2.3. Train Test Split
Objective: 
Data split into train/test to avoid data leakage. 

The tables will be first split before any analysis to minimize data 
leakage. <br>
*bureau_balance* does not have SK_ID_CURR, so it cannot be directly split into
 train/test by that key. However, rows are identified by SK_ID_BUREAU. This 
 method can also be used for *credit_card_balance*, *installments_payment*, 
 and *POS_CASH_balance*.<br>

**Split All Data into Train Test files -** All *test* files generated here are 
only used for kaggle submission

Split bureau by SK_ID_CURR

In [ ]:
bureau_train = bureau[bureau['SK_ID_CURR'].isin(sk_id_curr_train)]
bureau_test = bureau[bureau['SK_ID_CURR'].isin(sk_id_curr_test)]
duplicate_curr = (set(bureau_train['SK_ID_CURR'])
                  .intersection(set(bureau_test['SK_ID_CURR'])))
duplicate_bur = (set(bureau_train['SK_ID_BUREAU'])
                 .intersection(set(bureau_test['SK_ID_BUREAU'])))
print(f"duplicated SK_ID_CURR in bureau "
      f"train test tables: {duplicate_curr}"
      f"\nduplicated SK_ID_BUREAU in bureau train test tables: "
      f"{duplicate_bur}")

Split bureau_balance by SK_ID_BUREAU

In [ ]:
sk_id_bureau_train = bureau_train['SK_ID_BUREAU'].unique()
sk_id_bureau_test = bureau_test['SK_ID_BUREAU'].unique()

bureau_balance_train = bureau_balance[bureau_balance['SK_ID_BUREAU'].isin
(sk_id_bureau_train)]
bureau_balance_test = bureau_balance[bureau_balance['SK_ID_BUREAU'].isin
(sk_id_bureau_test)]

Split previous_application by SK_ID_CURR

In [ ]:
previous_application_train =(
    previous_application)[previous_application[
    'SK_ID_CURR'].isin(sk_id_curr_train)]
previous_application_test = (
    previous_application)[previous_application[
    'SK_ID_CURR'].isin(sk_id_curr_test)]
duplicate_curr = set(previous_application_train['SK_ID_CURR']).intersection(set
                                                         (previous_application_test['SK_ID_CURR']))
duplicate_bur = set(previous_application_train['SK_ID_PREV']).intersection(set
                                                         (previous_application_test['SK_ID_PREV']))
print(f"duplicated SK_ID_CURR in previous_application  "
      f"train test tables: {duplicate_curr}"
      f"\nduplicated SK_ID_PREV in previous_application train test tables: "
      f"{duplicate_bur}")

Split credit_card_balance, installments_payments, POS_CASH_balance by 
SK_ID_PREV

In [ ]:
sk_id_prev_train = previous_application_train['SK_ID_PREV'].unique()
sk_id_prev_test = previous_application_test['SK_ID_PREV'].unique()

credit_card_balance_train = (
    credit_card_balance)[credit_card_balance[('SK_ID_PREV')].isin(
    sk_id_prev_train)]
credit_card_balance_test = (
    credit_card_balance)[credit_card_balance['SK_ID_PREV'].isin(
    sk_id_prev_test)]

installments_payments_train = (
    installments_payments)[installments_payments['SK_ID_PREV'].isin(
    sk_id_prev_train)]
installments_payments_test = (
    installments_payments)[installments_payments['SK_ID_PREV'].isin(
    sk_id_prev_test)]

POS_CASH_balance_train = (
    POS_CASH_balance)[POS_CASH_balance['SK_ID_PREV'].isin(sk_id_prev_train)]
POS_CASH_balance_test = (
    POS_CASH_balance)[POS_CASH_balance['SK_ID_PREV'].isin(sk_id_prev_test)]

Save all test dataframes as .pkl files in a new folder called data_kaggle, 
as these files will only be used for kaggle submission.

In [ ]:
test_folder = "data_kaggle"
os.makedirs(test_folder, exist_ok=True)

data_pkl_folder = "data_pkl"
application_test_path = os.path.join(data_pkl_folder, "application_test.pkl")
application_test = pd.read_pickle(application_test_path)

test_dataframes = {
    'bureau_test': bureau_test,
    'bureau_balance_test': bureau_balance_test,
    'previous_application_test': previous_application_test,
    'credit_card_balance_test': credit_card_balance_test,
    'installments_payments_test': installments_payments_test,
    'POS_CASH_balance_test': POS_CASH_balance_test, 
    'application_test': application_test
}

for name, df in test_dataframes.items():
    file_path = os.path.join(test_folder, f"{name}.pkl")
    df.to_pickle(file_path)
    print(f"Saved {name} to {file_path}")

print("\nAll test DataFrames have been saved.")

**Split data into train/test for modeling**

In [ ]:
# Step 1: Extract SK_ID_CURR for train and test
data_train, data_test= train_test_split(
    application_train, test_size=0.2, random_state=42, 
    stratify=application_train['TARGET']
)

sk_id_curr_train_split = application_train.loc[data_train.index, 'SK_ID_CURR']
sk_id_curr_test_split = application_train.loc[data_test.index, 'SK_ID_CURR']

# Step 2: Filter related tables
def split_related_table(df, train_ids, test_ids, id_column):
    train_split = df[df[id_column].isin(train_ids)]
    test_split = df[df[id_column].isin(test_ids)]
    return train_split, test_split

related_tables = {
    'bureau': (bureau_train, 'SK_ID_CURR'),
    'bureau_balance': (bureau_balance_train, 'SK_ID_BUREAU'),
    'previous_application': (previous_application_train, 'SK_ID_CURR'),
    'credit_card_balance': (credit_card_balance_train, 'SK_ID_PREV'),
    'installments_payments': (installments_payments_train, 'SK_ID_PREV'),
    'POS_CASH_balance': (POS_CASH_balance_train, 'SK_ID_PREV'),
}

train_splits = {'data_train': data_train}
test_splits = {'data_test': data_test}

for table_name, (df, key_column) in related_tables.items():
    if key_column == 'SK_ID_CURR':
        train_split, test_split = split_related_table(df, sk_id_curr_train_split, sk_id_curr_test_split, key_column)
    elif key_column == 'SK_ID_BUREAU':
        sk_id_bureau_train_split = bureau_train[bureau_train['SK_ID_CURR'].isin(sk_id_curr_train_split)]['SK_ID_BUREAU']
        sk_id_bureau_test_split = bureau_train[bureau_train['SK_ID_CURR'].isin(sk_id_curr_test_split)]['SK_ID_BUREAU']
        train_split, test_split = split_related_table(df, sk_id_bureau_train_split, sk_id_bureau_test_split, key_column)
    elif key_column == 'SK_ID_PREV':
        sk_id_prev_train_split = previous_application_train[previous_application_train['SK_ID_CURR'].isin(sk_id_curr_train_split)]['SK_ID_PREV']
        sk_id_prev_test_split = previous_application_train[previous_application_train['SK_ID_CURR'].isin(sk_id_curr_test_split)]['SK_ID_PREV']
        train_split, test_split = split_related_table(df, sk_id_prev_train_split, sk_id_prev_test_split, key_column)
    
    train_splits[f"{table_name}_train"] = train_split
    test_splits[f"{table_name}_test"] = test_split

# Step 3: Save all splits to files
split_folder = "data_train_test_split"
os.makedirs(split_folder, exist_ok=True)

for table_name, df in train_splits.items():
    file_path = os.path.join(split_folder, f"{table_name}.pkl")
    df.to_pickle(file_path)
    print(f"Saved train split for {table_name} to {file_path}")

for table_name, df in test_splits.items():
    file_path = os.path.join(split_folder, f"{table_name}.pkl")
    df.to_pickle(file_path)
    print(f"Saved test split for {table_name} to {file_path}")

print("\nAll train-test splits have been saved.")

## 2.4. Aggregation
Objective: Aggregate data to the main table -> application_train

In [ ]:
encoder = OneHotEncoder(sparse=False, handle_unknown='ignore')

# Bureau Balance Encoding
def encode_with_onehot(df, column_to_encode, key_column, prefix):
    # Extract the column to encode and the keys
    keys = df[key_column]  # Preserve keys for later use
    column = df[[column_to_encode]]  # Column to encode
    
    # Fit and transform the encoder
    encoded_array = encoder.fit_transform(column)
    
    # Convert the encoded array to a DataFrame
    encoded_df = pd.DataFrame(
        encoded_array,
        columns=[f"{prefix}_{cat}" for cat in encoder.categories_[0]]
    )
    
    # Concatenate the keys and the encoded columns
    encoded_df[key_column] = keys.reset_index(drop=True)  # Add keys back
    return encoded_df

In [ ]:
bureau = aggregate_bureau_balance(bureau, bureau_balance)
previous_application = aggregate_previous_application(previous_application, credit_card_balance, installments_payments, POS_CASH_balance)
previous_application.head()

In [ ]:
# for bureau_balance
description.loc[description['Row'] == 'STATUS']

Status of Credit Bureau loan during the month (active, closed, DPD0-30, [C means closed, X means status unknown, 0 means no DPD, 1 means maximal did during month between 1-30, 2 means DPD 31-60, 5 means DPD 120+ or sold or written off ] )

In [ ]:
bureau_balance['STATUS'].unique()

In [ ]:
description.loc[description['Row'] == 'NAME_CONTRACT_STATUS']

In [ ]:
credit_card_balance['NAME_CONTRACT_STATUS'].unique()

# Improvements

1. Instead of aggregating data from other tables to the main table, maybe 
the model would benefit from predicting the probability of defaulting current 
loan based on previous bureau/loan data, which can then be aggregated to the
 main table.

changes made to tables:<br>
1. encode 'STATUS' and 'NAME_CONTRACT_STATUS'. one hot encoding for both